In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import torchreid


In [ ]:
from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000013_00000_v import VisDroneChild_uav0000013_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000013_00000_v', VisDroneChild_uav0000013_00000_v)

In [ ]:
from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000013_01073_v import VisDroneChild_uav0000013_01073_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000013_01073_v', VisDroneChild_uav0000013_01073_v)


In [ ]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000086_00000_v import VisDroneChild_uav0000086_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000086_00000_v', VisDroneChild_uav0000086_00000_v)


In [ ]:

from torchreid.data.datasets.video.VisDroneChilds.VisDroneChild_uav0000086_00000_v import VisDroneChild_uav0000086_00000_v
torchreid.data.register_video_dataset('VisDroneChild_uav0000086_00000_v', VisDroneChild_uav0000086_00000_v)


In [ ]:
datamanager = torchreid.data.VideoDataManager(
    root='D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-val',
    sources=['VisDroneChild_uav0000086_00000_v'],
    targets = ['VisDroneChild_uav0000086_00000_v'],
    height = 256,    #resizes images to this
    width=128,
    batch_size_train = 18,    #how many tracklets in a batch
    batch_size_test =18,    #how many tracklets in a batch
    seq_len = 2,    # how many frmaes in a tracklet
    num_instances = 3,    #how many tracklets per id in a batch
    train_sampler='CustomDatasetSampler'
)

In [ ]:
model = torchreid.models.build_model(
    name='osnet_ain_x0_25',
    num_classes=datamanager.max_train_pid+1,#КОСТЫЛЬ, возможно)
    loss='triplet',
    pretrained=False
    
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=300,
    gamma = 1
)


In [ ]:
engine = torchreid.engine.VideoTripletEngine(
    datamanager, model,optimizer,margin=100, weight_t=1, weight_x=0,scheduler=scheduler,
    pooling_method='avg'
)

In [ ]:
engine = torchreid.engine.VideoSoftmaxEngine(
    datamanager, model,optimizer,scheduler=scheduler,
    pooling_method='avg'
)

In [ ]:


engine.run(
    max_epoch=150,
    save_dir='log/osnet_ain_x0_25-triplet-visdrone',
    print_freq=5,  #if (self.batch_idx + 1) % print_freq == 0:, where for self.batch_idx, data in enumerate(self.train_loader):
    eval_freq=15
    #start_epoch=start_epoch,
    #test_only=True
    #visrank=True
    #start_epoch=start_epoch,
    
)
#loss_t - triplet loss
#loss_x - cross entropy loss
#validation on same as training - 100% results
#sampler selects tracklets to batches??
""" used like:
self.train_loader = torch.utils.data.DataLoader(
            trainset,
            sampler=build_train_sampler(
                trainset.train,
                train_sampler,
                batch_size=batch_size_train,
                num_instances=num_instances,
                num_cams=num_cams,
                num_datasets=num_datasets
            ),
            ...
        )
"""

"""
Если вообще не учить перед валидацией - 95+
Если хотябы одну эпоху на любом датасете будет хуже
В QUERY и GALLERY одну и тоже может попасть
"""
#HARD MINE YRIPLET LOSS distAP not considered
